In [4]:
import pandas as pd
import datetime

# 데이터 불러오기
cards = pd.read_csv("../data/cards_data.csv")
users = pd.read_csv("../data/users_data.csv")
mcc_codes = pd.read_json("../data/mcc_codes.json", typ='series')

# 전처리
cards['credit_limit'] = cards['credit_limit'].replace('[\$,]', '', regex=True).astype(float)
users['yearly_income'] = users['yearly_income'].replace('[\$,]', '', regex=True).astype(float)
users['total_debt'] = users['total_debt'].replace('[\$,]', '', regex=True).astype(float)

# 사기 탐지 함수
def detect_fraud(card, user):
    fraud_reasons = []

    # 1. 다크웹 노출 여부
    if card['card_on_dark_web'].strip().lower() == 'yes':
        fraud_reasons.append("Card exposed on dark web")

    # 2. 카드 유효기간 만료
    try:
        exp_date = datetime.datetime.strptime(card['expires'], "%m/%Y")
        if exp_date < datetime.datetime.now():
            fraud_reasons.append("Card expired")
    except:
        fraud_reasons.append("Invalid expiry date")

    # 3. 연령 70세 이상 + 고위험 MCC (시뮬레이션용 MCC 코드 7995: 도박)
    if user['current_age'] >= 70:
        if '7995' in mcc_codes.index:
            fraud_reasons.append("Elderly user with high-risk MCC: Betting")

    # 4. 위험 업종 코드 사용
    risky_mccs = ['7995', '4829', '5921']  # 도박, 송금, 주류 판매점 등
    if any(code in mcc_codes.index for code in risky_mccs):
        fraud_reasons.append("Risky MCC code detected")

    # 5. 신용한도 / 연소득 > 1.5
    if user['yearly_income'] > 0:
        ratio = card['credit_limit'] / user['yearly_income']
        if ratio > 1.5:
            fraud_reasons.append("Credit limit exceeds 150% of income")

    # 6. 신용점수 낮음
    if user['credit_score'] <= 600:
        fraud_reasons.append("Low credit score")

    return fraud_reasons

# 카드와 유저 데이터 연결
merged = cards.merge(users, left_on='client_id', right_on='id', suffixes=('_card', '_user'))

# 사기 탐지 적용
merged['fraud_flags'] = merged.apply(lambda row: detect_fraud(row, row), axis=1)

# 사기 의심 거래만 필터링
suspicious = merged[merged['fraud_flags'].apply(lambda x: len(x) > 0)]

# 결과 보기
print(suspicious[['card_number', 'client_id', 'credit_limit', 'yearly_income', 'credit_score', 'card_on_dark_web', 'expires', 'fraud_flags']])

           card_number  client_id  credit_limit  yearly_income  credit_score  \
0     4344676511950444        825       24295.0        59696.0           787   
1     4956965974959986        825       21968.0        59696.0           787   
2     4582313478255491        825       46414.0        59696.0           787   
3     4879494103069057        825       12400.0        59696.0           787   
4     5722874738736011        825          28.0        59696.0           787   
...                ...        ...           ...            ...           ...   
6141   300609782832003        185        6900.0        30942.0           779   
6142  4718517475996018        185        5700.0        30942.0           779   
6143  5929512204765914       1007        9200.0        54654.0           618   
6144  5589768928167462       1110       28074.0        65909.0           673   
6145  4994011318343994       1110       14400.0        65909.0           673   

     card_on_dark_web  expires     frau